In [1]:
import os
import pandas as pd
import joblib
from tqdm import tqdm
import datetime
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
%%time

a = pd.DataFrame({'filename1': os.listdir()})
b = a.filename1.str.startswith("security")
files = list(a[b].filename1)
final_df = pd.DataFrame()

for file in tqdm(files):
    df = joblib.load(file)
    if final_df.empty:
        final_df = df
    else:
        final_df = pd.concat([df, final_df])

100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:40<00:00,  2.77it/s]

Wall time: 40.3 s


In [3]:
final_df.sample(2)

,title,summary_x,links,link,id,guidislink,published,published_parsed,title_detail.type,title_detail.language,...,authorposts,postcategory,customexcerpt,categories,date,URL,authors_y,keywords,summary_y,text
313,How drones affect your threat model,<p>Most security leaders are unlikely to have ...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.csoonline.com/article/3568452/how-...,NaN,NaN,"Mon, 03 Aug 2020 03:00:00 -0700","(2020, 8, 3, 10, 0, 0, 0, 216, 0)",text/plain,None,...,NaN,NaN,NaN,,None,https://www.csoonline.com/article/3568452/how-...,[Jaikumar Vijayan],"[model, drone, security, unmanned, using, need...",Most security leaders are unlikely to have dro...,Most security leaders are unlikely to have dro...
346,"A look at Benchmark Analytics, which wants to ...","<A HREF=""https://www.bloomberg.com/news/articl...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",http://www.techmeme.com/200715/p41#a200715p41,http://www.techmeme.com/200715/p41#a200715p41,False,"Wed, 15 Jul 2020 16:20:01 -0400","(2020, 7, 15, 20, 20, 1, 2, 197, 0)",text/plain,None,...,NaN,NaN,NaN,NaN,2015-07-20 00:00:00,http://www.techmeme.com/200715/p41#a200715p41,[],"[jan, significant, amazon, deadline, look, com...",— - Amazon has told staff who can work from ho...,— - Amazon has told staff who can work from ho...


In [4]:
sum(list(final_df.URL == final_df.link))

41162

In [5]:
final_df2 = final_df[['title', 'summary_x', 'URL', 'published', 'keywords', 'summary_y', 'text' ]]

------

In [6]:
# the column 'published' has the time string.  Convert to datetime, as a timezone aware entry. 
# Fix EDT and PDT in advance

t1 = final_df2.published.apply(str)
t1 = [x.replace('EDT', '-0400') for x in t1]
t1 = [x.replace('PDT', '-0700') for x in t1]
t1 = [(pd.to_datetime(x)) for x in t1]
final_df2.published = t1

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
# Write a function to get just the data and time without timezone so it can be exported to Excel

def no_timezone(time_string):
    try:
        no_tz = datetime.strftime(time_string, "%Y-%m-%d %H:%M:%S")
    except:
        no_tz = 0
    return no_tz

In [8]:
# Get the published date (no timezone) in a new column
final_df2['published_date'] = final_df2.published.apply(no_timezone)

# Drop the published column 
final_df2.drop(columns = ['published'], inplace = True)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
len(final_df2)

41162

In [10]:
final_df2.sample(5)

,title,summary_x,URL,keywords,summary_y,text,published_date
240,"TikTok, WeChat Bans Not Crucial to US Security...","<p><strong><span style=""font-size: medium;""><s...",http://feedproxy.google.com/~r/Securityweek/~3...,"[china, trump, chinese, bans, crucial, wechat,...",The US bans on Chinese apps TikTok and WeChat ...,The US bans on Chinese apps TikTok and WeChat ...,2020-08-10 11:35:31
197,Improving Malicious Document Detection in Gmai...,"<span class=""byline-author"">Posted by Elie Bur...",http://feedproxy.google.com/~r/GoogleOnlineSec...,"[documents, users, improving, detection, scann...",The document analyzers are responsible for par...,\n\n\n\n\n\nSince the new scanner launched at ...,2020-02-25 12:00:00
293,Trump space speech in Florida likely to test a...,White House recognizes the role human spacefli...,https://arstechnica.com/?p=1679682,"[going, florida, nasa, test, crew, speech, pro...",After President Trump appointed a conservative...,After President Trump appointed a conservative...,2020-05-29 15:04:16
207,QakBot Banking Trojan Returned With New Sneaky...,A notorious banking trojan aimed at stealing b...,http://feedproxy.google.com/~r/TheHackersNews/...,"[sneaky, point, qbot, money, malspam, qakbot, ...","""An external operator can perform bank transac...",Using Hijacked Email Threads as Lures\n\nFrom ...,2020-08-27 02:59:37
292,9 tips to detect and prevent web shell attacks...,<p>One tool that bad guys use to go after your...,https://www.csoonline.com/article/3541614/9-ti...,"[networks, detect, files, windows, compare, ns...",One tool that bad guys use to go after your we...,One tool that bad guys use to go after your we...,2020-05-13 03:00:00


In [9]:
# print((final_df2.applymap(type) == list).all())

In [10]:
# print(final_df2.applymap(type))

In [11]:
# a = ['one', 'two', 'three']
# b = ','.join(a)

In [12]:

final_df2.keywords = [','.join(a) for a in final_df2.keywords.copy(deep=True)]

In [13]:
print('Length before drop_duplicates', len(final_df2))
final_df2 = final_df2.drop_duplicates()
print('Length after drop_duplicates', len(final_df2))


Length before drop_duplicates 41162
Length after drop_duplicates 38676


In [14]:
final_df2.text.nunique()
# final_df2.text2=final_df2.text

10369

In [15]:
final_df3 = final_df2.groupby('text').head(1)
print(len(final_df3), 'articles in final_df3')

10369 articles in final_df3


In [16]:
final_df3.reset_index(drop=True, inplace=True)

In [17]:
final_df3


,title,summary_x,URL,keywords,summary_y,text,published_date
0,Friday Squid Blogging: Morning Squid,"Asa ika means ""morning squid"" in Japanese. As ...",https://www.schneier.com/blog/archives/2020/09...,"w,r,i,t,i,n,g,,,p,e,r,s,o,n,a,l,,,w,o,r,k,i,n,...",About Bruce SchneierI am a public-interest tec...,About Bruce Schneier\n\nI am a public-interest...,2020-09-04 21:53:54
1,Insider Attack on the Carnegie Library,"Greg Priore, the person in charge of the rare ...",https://www.schneier.com/blog/archives/2020/09...,"p,e,r,e,n,n,i,a,l,,,c,a,r,n,e,g,i,e,,,t,r,u,s,...",Insider Attack on the Carnegie LibraryGreg Pri...,Insider Attack on the Carnegie Library\n\nGreg...,2020-09-02 12:02:03
2,North Korea ATM Hack,The US Cybersecurity and Infrastructure Securi...,https://www.schneier.com/blog/archives/2020/09...,"c,y,b,e,r,s,e,c,u,r,i,t,y,,,c,i,s,a,,,s,c,h,n,...",North Korea ATM HackThe US Cybersecurity and I...,North Korea ATM Hack\n\nThe US Cybersecurity a...,2020-09-01 11:17:39
3,US Postal Service Files Blockchain Voting Patent,The US Postal Service has filed a patent on a ...,https://www.schneier.com/blog/archives/2020/08...,"m,a,i,l,,,v,o,t,e,s,,,b,l,o,c,k,c,h,a,i,n,,,s,...",US Postal Service Files Blockchain Voting Pate...,US Postal Service Files Blockchain Voting Pate...,2020-08-28 11:40:08
4,Cory Doctorow on The Age of Surveillance Capit...,Cory Doctorow has writtten an extended rebutta...,https://www.schneier.com/blog/archives/2020/08...,"u,n,i,q,u,e,,,u,n,e,t,h,i,c,a,l,,,s,o,c,i,e,t,...",Cory Doctorow on The Age of Surveillance Capit...,Cory Doctorow on The Age of Surveillance Capit...,2020-08-27 11:33:31
...,...,...,...,...,...,...,...
10364,Amazon confirms a major outbreak of COVID-19 a...,"<A HREF=""https://www.businessinsider.com/amazo...",http://www.techmeme.com/200423/p38#a200423p38,"v,i,d,e,o,,,p,a,s,t,,,s,e,c,u,r,i,t,y,,,w,a,r,...",— Zoom Video Communications Inc. has been lamb...,— Zoom Video Communications Inc. has been lamb...,2020-04-23 19:55:23
10365,The coronavirus is accelerating the shift to a...,"<A HREF=""https://www.ft.com/content/990e89de-8...",http://www.techmeme.com/200423/p33#a200423p33,"s,h,i,f,t,,,f,i,n,a,n,c,i,a,l,,,r,i,g,h,t,,,w,...",— Miro is a company in the right place at the ...,— Miro is a company in the right place at the ...,2020-04-23 17:30:02
10366,Doctors are using AI to triage covid-19 patien...,Rizwan Malik had always had an interest in AI....,https://www.technologyreview.com/2020/04/23/10...,"p,a,n,d,e,m,i,c,,,h,e,a,l,t,h,,,s,y,s,t,e,m,,,...","The pandemic, in other words, has turned into ...",The Royal Bolton Hospital is among a growing n...,2020-04-23 14:00:00
10367,The race to save the first draft of coronaviru...,"Eight years ago, Suleika Jaouad was alone in a...",https://www.technologyreview.com/2020/04/21/99...,"s,a,v,e,,,p,a,n,d,e,m,i,c,,,l,i,f,e,,,i,n,t,e,...",But getting the internet to archive as much as...,"Within a week, Blair’s tweet got the attention...",2020-04-21 09:00:00


In [18]:
final_df3.sample(3)

,title,summary_x,URL,keywords,summary_y,text,published_date
3682,U.S. Imposes Sanctions on 11 Chinese Companies...,"The move, which affects suppliers to major int...",https://www.nytimes.com/2020/07/20/business/ec...,"c,o,m,p,a,n,i,e,s,,,i,m,p,o,s,e,s,,,1,1,,,s,a,...",WASHINGTON — The Trump administration on Monda...,WASHINGTON — The Trump administration on Monda...,2020-07-21 00:29:16
9041,"More Than 8,000 Unsecured Redis Instances Foun...","<img width=""200"" height=""125"" src=""https://blo...",http://feeds.trendmicro.com/~r/Anti-MalwareBlo...,"d,a,t,a,,,c,o,m,m,a,n,d,,,i,n,s,t,a,n,c,e,s,,,...","In a previous case, a fake ransomware called F...",By David Fiser (Security Researcher)\n\nWe dis...,2020-04-02 12:57:40
4215,US the Primary Target of ‘Significant’ Cyber-A...,The US has faced 156 major cyber-attacks over ...,https://www.infosecurity-magazine.com:443/news...,"c,o,u,n,t,r,i,e,s,,,a,t,t,a,c,k,s,,,s,p,e,c,o,...",The US experienced far more ‘significant’ cybe...,The US experienced far more ‘significant’ cybe...,2020-07-15 08:20:00


In [19]:
# sum(final_df3.title.str.contains("Corona|corona|coronavirus|covid|Covid|cyber"))

In [20]:
final_df3.to_excel('temp.xlsx',index=False)

In [23]:
# Define a function for True/False for corona depending on if any of the corona related
# strings is found in a piece of text

def corona(text):
    corona_story_strings = ['covid', 'corona', 'sars', 'virus', 'coronavirus', 'vaccine']
    return any(x in text for x in corona_story_strings)
    
    


In [24]:
final_df3['corona'] = final_df3.text.apply(corona)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
joblib.dump(final_df3, 'final_df3.pkl')

['final_df3.pkl']

In [29]:
final_df3.shape

(6036, 8)

In [26]:
final_df3.corona.value_counts()

False    4779
True     1257
Name: corona, dtype: int64

In [27]:
final_df3.query('corona == True')

,title,summary_x,URL,keywords,summary_y,text,published_date,corona
20,Evilnum Group targets European and British fin...,<p>A threat actor tracked as Evilnum targeted ...,https://securityaffairs.co/wordpress/105784/cy...,"group,evilnum,companies,european,attacks,used,...",A threat actor tracked as Evilnum targeted fin...,A threat actor tracked as Evilnum targeted fin...,2020-07-11 04:45:03,True
27,15 billion credentials available in the cyberc...,<p>More than 15 billion username and passwords...,https://securityaffairs.co/wordpress/105698/ha...,"file,accounts,15,report,access,cybercrime,aver...",More than 15 billion username and passwords ar...,More than 15 billion username and passwords ar...,2020-07-09 14:05:45,True
30,15K Billion Compromised Credentials Available ...,"There are currently 15… <a href=""https://healt...",https://healthitsecurity.com/news/15k-billion-...,"hacker,accounts,compromised,attacks,researcher...",The number of stolen credentials for sale on t...,"By Jessica Davis\n\nJuly 09, 2020 - There are ...",0,True
32,"Microsoft Sues, Now Controls COVID-19 Phishing...","The US District Court for… <a href=""https://he...",https://healthitsecurity.com/news/microsoft-su...,"case,campaign,attacks,order,court,microsoft,do...",The judge issued a civil court order that allo...,"By Jessica Davis\n\nJuly 08, 2020 - The US Dis...",0,True
33,"NSA Shares Guide to Securing IPSec VPNs, Telew...","The National Security… <a href=""https://health...",https://healthitsecurity.com/news/nsa-shares-g...,"telework,cnssp,shares,sites,nsa,security,crypt...","By Jessica DavisJuly 07, 2020 - The National S...","By Jessica Davis\n\nJuly 07, 2020 - The Nation...",0,True
...,...,...,...,...,...,...,...,...
6029,Nextdoor and Walmart partner on a new neighbor...,Neighborhood social network Nextdoor and Walma...,http://feedproxy.google.com/~r/Techcrunch/~3/_...,"offer,groups,feature,nextdoor,help,neighbors,n...",Neighborhood social network Nextdoor and Walma...,Neighborhood social network Nextdoor and Walma...,2020-04-23 16:22:09,True
6030,Daily Crunch: There’s a major iPhone email sec...,Apple plans to fix an iPhone email security bu...,http://feedproxy.google.com/~r/Techcrunch/~3/a...,"google,bug,daily,firm,security,extra,techcrunc...",Apple plans to fix an iPhone email security bu...,Apple plans to fix an iPhone email security bu...,2020-04-23 16:10:28,True
6033,Doctors are using AI to triage covid-19 patien...,Rizwan Malik had always had an interest in AI....,https://www.technologyreview.com/2020/04/23/10...,"pandemic,health,system,triage,using,coronaviru...","The pandemic, in other words, has turned into ...",The Royal Bolton Hospital is among a growing n...,2020-04-23 14:00:00,True
6034,The race to save the first draft of coronaviru...,"Eight years ago, Suleika Jaouad was alone in a...",https://www.technologyreview.com/2020/04/21/99...,"save,pandemic,life,internet,history,coronaviru...",But getting the internet to archive as much as...,"Within a week, Blair’s tweet got the attention...",2020-04-21 09:00:00,True


In [115]:
final_df3.query('corona == True').to_excel('temp.xlsx')

In [1]:
!pip install tabloo


  Created wheel for tabloo: filename=tabloo-0.0.13-py3-none-any.whl size=2329707 sha256=895c5304f3c3851e4312800212b57b46f394d8d082b00096e5e69f94113fbb37
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ac\d6\6d\33d33c6bb6bf642ba5e0d08225a1c025952391f0a9d4fb0e95
Successfully built tabloo


NameError: name 'final_df3' is not defined